In [8]:
import numpy as np


def env():
    # 定义系统的A和B矩阵
    A = np.array([[0, 10], 
                  [-10, 0]])  # 系统矩阵 A
    B = np.array([[0],
                  [1]])  # 系统矩阵 B
    return A
    
# 时间间隔，用于环境中的数据采样
def init_time_intervals(n, m):
    delta_t = 0.01
    return np.arange(0, (n*n*100+1)*delta_t, delta_t)  # 样本时间间隔
    
def init_state():
    # 初始化算法参数
    
    n = 2  # 系统状态维度
    m = 1  # 控制输入维度
    
    # # 定义初始化K矩阵
    # K_i_0 = np.zeros((m, n))
    # #print(x_0)
    
    # # 初始矩阵 P_i^j 
    # P_i_j = np.eye(n)  
    return n, m



In [9]:
import numpy as np
def integrate_2d_array(x_i, x_i_d, delta_t):
    return x_i + delta_t * x_i_d

# 采样
def sample(n, m):
    # 获取A，B
    A = env()

    # 时间间隔，用于环境中的数据采样
    time_intervals = init_time_intervals(n, m)

    # 随机初始化状态x_i
    seed = 1
    x_i = np.random.rand(n, 1)   
    # x_i = np.array([[1],
    #                 [2],
    #                 [3],
    #                 [4]])
    # 数据收集初始化，存放x_i的每个采样状态，用于计算 Ψ_i、Φ_i、Π_i 矩阵
    x_data = x_i

    # 采样循环
    for t in range(1, len(time_intervals)):
        # 生成一个 大小2x1 的 Gaussian 噪声数组
        gaussian_noise = np.random.normal(0, 1, (n, 1))

        # 计算控制输入 ξ_i,大小m*1和下一个状态的导数x_i_d,大小n*1,
        # xi_i = K @ x_i + gaussian_noise
        x_i_d = A @ x_i
        # x_i_d = A @ (x_i + gaussian_noise)

        # 求下一个状态,n*1
        delta_t = time_intervals[t] - time_intervals[t - 1]
        x_i_next = integrate_2d_array(x_i, x_i_d, delta_t)

        # 收集状态,添加一列数据
        x_data = np.hstack((x_data, x_i_next))
        
        # 更新至下一个状态
        x_i = x_i_next
        #print(x_i)

    # # 删除掉初始时全为0的第一列元素
    # x_data = np.delete(x_data, 0, axis=1) 
    # xi_data = np.delete(xi_data, 0, axis=1) 
    print(f"x_data.shape:{x_data.shape}") 
    # print(f"x_data.shape[1]-1:{x_data.shape[1]-1}") 
    
    print(f"x_data:{x_data}") 
    # plot_x_data(x_data)
    return x_data
# n = 4
# m = 2
# time_intervals = np.arange(0, n*(n+1)/2+n*m)  # 样本时间间隔
# print(time_intervals)

In [10]:
# 计算Ψ_i矩阵
def get_Psi_i(n, x_data, ones_vector):
    # 初始化Ψ_i矩阵
    Psi_i = np.zeros((n*n, 1))
    # print(f"Psi_i.shape{Psi_i.shape}")
    # print(f"Psi_i{Psi_i}")

    # 如果是按开始时间算就是0~17列，如果是按结束时间算就是1~18列
    column = x_data.shape[1] - 1
    for t in range(column):
        x_i = x_data[:, t] # 提出一列元素变为一维数组
        x_i = x_i.reshape(-1,1) # 转为二维数组4*1
        #print(x_i.shape)
        Psi_i = np.hstack((Psi_i, np.kron(ones_vector, x_i)))
    Psi_i = np.delete(Psi_i, 0, axis=1) # 删除掉初始时全为0的第一列元素
    Psi_i = Psi_i.T
    print(f"Psi_i.shape{Psi_i.shape}")
    return Psi_i

In [11]:

def integrate_2d_array_2(x, delta_t):
    return delta_t * x


# 计算Γ_i 矩阵
def get_Gamma(n, m, x_data, ones_vector):
    # # 采样
    # x_data, xi_data = sample(n, m, K)
    
    # 时间间隔，用于环境中的数据采样
    time_intervals = init_time_intervals(n, m)

    # 初始化Γ_i矩阵
    Gamma_i = np.zeros((n*n, 1))
    # 计算 Γ_i 矩阵
    column = x_data.shape[1]
    for t in range(1,column):
        delta_t = time_intervals[t] - time_intervals[t-1]
        x_i = x_data[:, t] # 提出一列元素变为一维数组
        x_i = x_i.reshape(-1,1) # 转为二维数组4*1
        x_i_kron = np.kron(ones_vector, x_i)
        x_i_kron_intergral = integrate_2d_array_2(x_i_kron, delta_t)
        Gamma_i = np.hstack((Gamma_i, x_i_kron_intergral))
    Gamma_i = np.delete(Gamma_i, 0, axis=1) # 删除掉初始时全为0的第一列元素
    Gamma_i = Gamma_i.T
    print(f"Gamma_i.shape:{Gamma_i.shape}") #18*10


    return Gamma_i


In [12]:

# 检查矩阵 Γ_i 是否满足秩条件
def check_Gamma(n, Gamma_i):
    if np.linalg.matrix_rank(Gamma_i) >= n*n:
        print("Gamma_i：{Gamma_i} 满足秩条件。")
    else:
        print("Gamma_i：{Gamma_i} 不满足秩条件。")
        print(f"np.linalg.matrix_rank(Gamma_i):{np.linalg.matrix_rank(Gamma_i)}")


In [13]:
# 将一列的二维数组变为方阵
def column_vector_to_square_matrix(column_vector):
    length = column_vector.shape[0]
    side_length = int(np.sqrt(length))
    if side_length ** 2 != length:
        raise ValueError("The length of the column vector must be a perfect square to reshape into a square matrix.")
    return column_vector.reshape(side_length, side_length)

def solve_A(max_iters=1000000000):
    n, m= init_state()
    # 初始化 ones_vector,I
    ones_vector =  np.ones((n, 1))
    I = np.ones((n * n, 1))

    # 采样
    x_data = sample(n, m)
    # print(f"x_data:{x_data.shape}")
    # plot_x_data(x_data)

    #获取相关矩阵信息
    Psi_i = get_Psi_i(n, x_data, ones_vector)
    gamma_i = get_Gamma(n, m, x_data, ones_vector)
    print(f"gamma_i{gamma_i}")
    # 检查矩阵 Γ_i 是否满足秩条件
    check_Gamma(n,gamma_i)
    
    # 计算 vec(A)
    vec_A = (np.linalg.inv(gamma_i.T @ gamma_i)) @ gamma_i.T @ Psi_i @ I

    # 将 vec(A) 还原为矩阵 A
    A = column_vector_to_square_matrix(vec_A)

    return A



In [14]:
A_final = solve_A()
print(f"A_final:{A_final}")

x_data.shape:(2, 401)
x_data:[[ 1.43851634e-01  1.61578702e-01  1.77867254e-01  1.92540019e-01
   2.05434112e-01  2.16402804e-01  2.25317155e-01  2.32067478e-01
   2.36564630e-01  2.38741106e-01  2.38551937e-01  2.35975356e-01
   2.31013256e-01  2.23691403e-01  2.14059416e-01  2.02190516e-01
   1.88181022e-01  1.72149622e-01  1.54236413e-01  1.34601707e-01
   1.13424637e-01  9.09015497e-02  6.72442162e-02  4.26778673e-02
   1.74390761e-02 -8.22649371e-03 -3.40664543e-02 -5.98241499e-02
  -8.52411810e-02 -1.10059971e-01 -1.34026348e-01 -1.56892126e-01
  -1.78417641e-01 -1.98374234e-01 -2.16546651e-01 -2.32735326e-01
  -2.46758534e-01 -2.58454389e-01 -2.67682658e-01 -2.74326384e-01
  -2.78293283e-01 -2.79516918e-01 -2.77957620e-01 -2.73603153e-01
  -2.66469110e-01 -2.56599036e-01 -2.44064270e-01 -2.28963514e-01
  -2.11422115e-01 -1.91591081e-01 -1.69645826e-01 -1.45784661e-01
  -1.20227036e-01 -9.32115657e-02 -6.49938246e-02 -3.58439678e-02
  -6.04417282e-03  2.41140619e-02  5.43327383e-

LinAlgError: Singular matrix

In [ ]:
# import numpy as np

# # 假设我们有采样数据 x(t)，采样时间间隔为 delta_t
# # x_samples 是一个二维数组，每一列表示不同时间的采样
# x_samples = np.array([
#     [1.0, 2],
#     [1.1, 2.1],
#     [1.2, 2.4],
#     [1.3, 2.6],
#     # ... 更多采样数据
# ])

# delta_t = 0.1  # 采样时间间隔

# # 定义 n*1 维的全 1 向量
# n = x_samples.shape[0]
# ones_vector = np.ones((n, 1))

# # 计算积分 \int_t^(t+\delta t) 1 \otimes x d\tau
# def compute_integral_1_otimes_x(x_samples, delta_t):
#     n = x_samples.shape[1]  # 状态向量的维度
#     num_samples = x_samples.shape[0]  # 采样点数量
    
#     integral_result = np.zeros((n, n))
#     for i in range(num_samples - 1):
#         x_t = x_samples[i]
#         x_next = x_samples[i + 1]
        
#         # 使用梯形法则近似积分
#         integral = 0.5 * (np.outer(x_t, x_t) + np.outer(x_next, x_next)) * delta_t
#         integral_result += integral
    
#     return integral_result

# # 计算 \Phi_i 和 F_i
# def compute_phi_and_f(x_samples, delta_t):
#     n = x_samples.shape[1]
#     num_samples = x_samples.shape[0]
    
#     phi_list = []
#     f_list = []
#     for i in range(num_samples - 1):
#         x_t = x_samples[i].reshape(-1, 1)  # 将 x_t 转换为列向量
#         x_next = x_samples[i + 1].reshape(-1, 1)  # 将 x_next 转换为列向量
        
#         # 计算 \Phi_i 和 F_i
#         phi = np.kron(ones_vector.T, x_t.T) * delta_t
#         f = (x_next - x_t).flatten()
        
#         phi_list.append(phi)
#         f_list.append(f)
    
#     return np.vstack(phi_list), np.hstack(f_list)

# # 使用最小二乘法求解 vec(A)
# Phi, F = compute_phi_and_f(x_samples, delta_t)
# vec_A, _, _, _ = np.linalg.lstsq(Phi, F, rcond=None)

# # 将 vec(A) 还原为矩阵 A
# n = x_samples.shape[1]
# A = vec_A.reshape((n, n))

# print("矩阵 A:")
# print(A)
